<a href="https://colab.research.google.com/github/bominwang/Bayesian-neural-network/blob/Engineering-application/BNN_for_HB2_SurfacePressure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

利用UBER开源的PYRO实现全连接层贝叶斯神经网络

In [34]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import torch
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
import matplotlib
from matplotlib import pyplot as plt
import tqdm
from tqdm import trange

In [36]:
!pip3 install pyro-ppl 
import pyro
import pyro.nn
from pyro.nn import PyroModule
from pyro.nn import PyroSample
from pyro.distributions import Normal, Uniform
from pyro.optim import Adam
import pyro.infer
from pyro.infer import SVI, Trace_ELBO
from pyro.infer import autoguide

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
def building_dataset(ins_path, lab_path, train_volume, device):
  ins = pd.read_csv(ins_path)
  labs = pd.read_csv(lab_path)
  
  ins = np.array(ins)
  labs = np.array(labs)
  labs = labs[:,1].reshape(-1,10)

  data_volume = ins.shape[0]
  test_volume = data_volume - train_volume
  
  data_index = list(range(data_volume))
  train_index = random.sample(range(0, data_volume), train_volume)
  test_index = np.setdiff1d(data_index, train_index)

  train_data_ins = torch.from_numpy(ins[train_index, :])
  train_data_labs = torch.from_numpy(labs[train_index, :])
  
  test_data_ins = torch.from_numpy(ins[test_index, :])
  test_data_labs = torch.from_numpy(labs[test_index, :])
  
  train_set = TensorDataset(train_data_ins, train_data_labs)
  train_set = DataLoader(train_set, batch_size = train_volume, shuffle = True)
  
  test_set = TensorDataset(test_data_ins, test_data_labs)
  test_set = DataLoader(test_set, batch_size = test_volume, shuffle = True)

  return train_set, test_set

定义神经网络

In [97]:
class bnn(PyroModule):
  def __init__(self):
    super(bnn, self).__init__()

    # 在利用PyroModule构建随机网络时，不能使用nn.Sequential来定义输入输出间的传递，因为利用nn.Sequential进行封装后，无法直接利用hidden_layer.weight\bias来调取权重和参数
    self.hidden_layer1 = PyroModule[nn.Linear](2, 100)
    self.hidden_layer1.weight = PyroSample(Normal(0.0, 1.0).expand([100, 2]).to_event(2))
    self.hidden_layer1.bias = PyroSample(Normal(0.0, 1.0).expand([100]).to_event(1))
    
    self.hidden_layer2 = PyroModule[nn.Linear](100, 100)
    self.hidden_layer2.weight = PyroSample(Normal(0.0, 1.0).expand([100, 100]).to_event(2))
    self.hidden_layer2.bias = PyroSample(Normal(0.0, 1.0).expand([100]).to_event(1))

    self.hidden_layer3 = PyroModule[nn.Linear](100, 10)
    self.hidden_layer3.weight = PyroSample(Normal(0.0, 1.0).expand([10, 100]).to_event(2))
    self.hidden_layer3.bias = PyroSample(Normal(0.0, 1.0).expand([10]).to_event(1))

    self.relu = nn.ReLU()
  
  def forward(self, x, y = None):
    x = x.reshape(-1, 2)
    x = self.relu(self.hidden_layer1(x))
    x = self.relu(self.hidden_layer2(x))
    
    mu = self.hidden_layer3(x).squeeze()

    # 去除size为1的维度，包括行和列。当维度大于等于2时，squeeze()无作用.eg : squeeze(0)代表若第一维度值为1则去除第一维度
    
    sigma = pyro.sample('sigma', Uniform(0.0, 1.0).expand([10]))

    with pyro.plate('data', x.shape[0]):
      obs = pyro.sample('obs', Normal(mu, sigma), obs = y)
    
    return mu

In [98]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = bnn()
guide = autoguide.AutoDiagonalNormal(model)
adam = Adam({'lr' : 1e-3})
svi = SVI(model, guide, adam, loss = Trace_ELBO())

pyro.clear_param_store()

loss = []
epoch = trange(20)
ins_path = 'drive/MyDrive/Colab Notebooks/bm_wang/data/HB2/inv/inv_input_data.csv'
labs_path = 'drive/MyDrive/Colab Notebooks/bm_wang/data/HB2/inv/inv_output_data.csv'
train_set, test_set = building_dataset(ins_path, labs_path, 100, device)

for i in epoch:
  for index, (ins, labs) in enumerate(train_set):
    ins = ins.reshape(-1,2).float()
    labs = labs.reshape(10,-1).float()
    l = svi.step(ins, labs)





  0%|          | 0/20 [00:00<?, ?it/s]


ValueError: ignored